Text einlesen
 - url angeben
 - echten header statt python, sonst mag es der gerufene server nicht
 - umcodieren der deutschen Umlaute nach UTF 8

 Text in eine Datei schreiben

In [1]:
#
import urllib.request
import re
from collections import Counter


In [19]:
# Buchkapitel lesen
def lese_kapitel(buch, kptnr):
    # Parameter
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; UTF8; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    
    # open a connection to a URL using urllib
    # url = "https://www.bibleserver.com/LUT/Markus1"
    if (buch == "Mat"):
        url = "https://www.bibleserver.com/LUT/Matth%C3%A4us" + kptnr  #übersetzer'.encode('punycode').decode() 
    else:  
        url = "https://www.bibleserver.com/LUT/"+ buch + "/" + kptnr
    # print(url)
      
    request=urllib.request.Request(url,None,headers) #The assembled request

    response = urllib.request.urlopen(request)

    data = response.read().decode(response.headers.get_content_charset()) # The data u need # decode nachUTF8 wegen Umlauten....

    # relevanten Teil rausschneiden
    phs = re.findall(r'<span class="verse-content--hover">(.*?)</span>',str(data))
    # print(phs)
    doc = ""
    for eachP in phs:
        doc = doc + " " + eachP

    return doc

Jetzt kommt die eigentliche Textanalyse mit Spacy

In [5]:
import spacy.cli
spacy.cli.download("de_core_news_md")

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_md')


In [3]:
 # spacy.load('de_core_news_md')

In [6]:
import spacy
nlp = spacy.load('de_core_news_md')

In [7]:
def token_text(doc):
    # tokenizer
    text= nlp(doc)

    #remove stopwords and punctuations
    words = [token.lemma_ for token in text if token.is_stop != True and token.is_punct != True]
    word_freq = Counter(words)
    # Was sind die top 19?
    common_words = word_freq.most_common(10)
    return common_words

In [8]:
def search_maxkap(buch):
    kapmax_liste = [("Markus",18),("Matthäus", 28), ("Lukas", 24) , ("Johannes", 21)]
    for i in range(len(kapmax_liste)):
        if kapmax_liste[i][0] == buch:
            return kapmax_liste[i][1]
        #print(kapmax_liste[i][0])       
    return 0

# k = search_maxkap("Lukas")
# print(k)

def check_buch(buch,kapmax):
        
    buchtext = ""
    kap      = 1
    while kap <= kapmax: 
        # print(kap)
        kapitel = lese_kapitel(buch,str(kap))
        buchtext = buchtext + kapitel
        kap = kap + 1
    
    top10   = token_text(buchtext)
    return top10


In [21]:
erg = check_buch("Mat",18)
print(erg)
erg = check_buch("Markus",28)
print(erg)
erg = check_buch("Lukas",24)
print(erg)
erg = check_buch("Johannes",21)
print(erg)
erg = check_buch("Apostelgeschichte",28)
print(erg)

[('sprechen', 130), ('Jesus', 87), ('Vater', 47), ('sage', 43), ('hören', 42), ('mein', 42), ('zeugen', 39), ('sehen', 37), ('Volk', 36), ("'s", 35)]
[('sprechen', 183), ('Jesus', 130), ('sehen', 89), ('glauben', 88), ('gehen', 70), ('Hand', 66), ("'s", 62), ('sprachen', 59), ('Grab', 55), ('kommen', 50)]
[('sprechen', 297), ('Sohn', 123), ('Jesus', 98), ('sehen', 74), ('Gott', 64), ('Herr', 62), ('sprachen', 61), ('hören', 59), ('Volk', 57), ('Vater', 56)]
[('Jesus', 248), ('sprechen', 187), ('Vater', 136), ('sehen', 101), ('glauben', 88), ('Welt', 79), ('antworten', 79), ('sprachen', 72), ('Jude', 69), ('kommen', 67)]


In [ ]:
# ner
labels = set([w.label_ for w in text.ents]) 
for label in labels: 
    entities = [e.string for e in text.ents if label==e.label_] 
    entities = list(set(entities)) 
    print( label,entities)

In [22]:
erg = check_buch("Apostelgeschichte",28)
print(erg)

[('sprechen', 150), ('Paulus', 137), ('Mann', 109), ('hören', 88), ('kommen', 88), ('Volk', 85), ('lassen', 82), ('Jude', 74), ('Geist', 72), ('sehen', 69)]
